## Merck Question/SQL Data Exploration and Preprocessing

The notebook loads the original question/sql data from Merck, and aggregates the different sheets (ground-truth SQL and GPT4 responses) in the excel file into a single CSV file.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

# !pip install openpyxl

In [2]:
DATA_PATH = "s3://aws-gaiic-merck-text2sql/data/synpuf_query_new.xlsx"
COMBINED_DATA_PATH = "s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined.csv"

QA_SHEET_NAME = 'query'
GPT4_SHEET_NAME = 'Sheet1'
GPT4_COLUMNS=['GPT-4 response', 'Comment ', 'Revised GPT-4 response']

COLUMN_MAPPINGS = {'Table reference': 'qa_id', 'Description': 'question', 'Synpuf query': 'query', 'Difficulty':'difficulty_level',
       'Genreralization': 'code_list_needed', 'Input table': 'code_list_table', 'GPT-4 response': 'gpt4_response', 'Revised GPT-4 response': 'gpt4_revised_response', 'Comment ': 'gpt4_followup_input'}

In [4]:
df_qa = pd.read_excel(DATA_PATH, sheet_name = QA_SHEET_NAME)
print(df_qa.shape)
df_qa.head()

(40, 6)


Table reference  \
0              t1   
1              t2   
2              t3   
3              t4   
4              t5   

                                                                                                                                                                                                                                                          Synpuf query  \
0                                                                                                                                                                                          select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;   
1                                                                                                                                                                                           select count(distinct desynpuf_id) from rwdex_raw_synpuf.inpatient_claims;   
2                                                                                                                                           select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'   
3                                                                                                                                          select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'   
4  select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims \nwhere clm_from_dt between '2015-01-01' and '2016-01-01' \nunion select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims\nwhere clm_admsn_dt between '2015-01-01' and '2016-01-01'    

                               Description  Difficulty Genreralization  \
0          count the number of outpatients         1.0             NaN   
1           count the number of inpatients         1.0             NaN   
2       select all inpatients in year 2005         2.0             Yes   
3      select all outpatients in year 2006         2.0             Yes   
4  return total patients from 2015 to 2016         3.0             NaN   

  Input table  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN

In [5]:
df_gpt4 = pd.read_excel(DATA_PATH, sheet_name = GPT4_SHEET_NAME)
print(df_gpt4.shape)
df_gpt4.head()

(40, 8)


Table reference  \
0              t1   
1              t2   
2              t3   
3              t4   
4              t5   

                                                                                                                                                                                                                                                          Synpuf query  \
0                                                                                                                                                                                          select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;   
1                                                                                                                                                                                           select count(distinct desynpuf_id) from rwdex_raw_synpuf.inpatient_claims;   
2                                                                                                                                           select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'   
3                                                                                                                                          select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'   
4  select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims \nwhere clm_from_dt between '2015-01-01' and '2016-01-01' \nunion select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims\nwhere clm_admsn_dt between '2015-01-01' and '2016-01-01'    

                                    Description  \
0               count the number of outpatients   
1       count the number of distinct inpatients   
2   select all distinct inpatients in year 2005   
3  select all distinct outpatients in year 2006   
4       return total patients from 2015 to 2016   

                                                                                                                                                      GPT-4 response  \
0                                                                                      SELECT COUNT(*) AS outpatient_count\nFROM rwdex_raw_synpuf.outpatient_claims;   
1                                                            SELECT COUNT(DISTINCT desynpuf_id) AS distinct_inpatient_count\nFROM rwdex_raw_synpuf.inpatient_claims;   
2                          SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_synpuf.inpatient_claims\nWHERE clm_admsn_dt >= '2005-01-01' AND clm_admsn_dt <= '2005-12-31';   
3                           SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_synpuf.outpatient_claims\nWHERE clm_from_dt >= '2006-01-01' AND clm_from_dt <= '2006-12-31';   
4  SELECT COUNT(DISTINCT desynpuf_id) AS total_patients\nFROM rwdex_raw_synpuf.outpatient_claims\nWHERE clm_from_dt >= '2015-01-01' AND clm_from_dt <= '2016-12-31';   

                                                    Comment   \
0                    I want the distinct numbers of patients   
1                                                        NaN   
2                                                        NaN   
3                                                        NaN   
4  I want results from two tables: outpatient and inpatients   

                                                                                                                                                                                                                                                                                                                                                                                       Revised GPT-4 response  \
0                                                                                                                                                                               

In [6]:
#Combine both ground truth and gpt-4 outputs
df = pd.concat([df_qa, df_gpt4[GPT4_COLUMNS]], axis=1)
print(df.shape)
df.head()

(40, 9)


Table reference  \
0              t1   
1              t2   
2              t3   
3              t4   
4              t5   

                                                                                                                                                                                                                                                          Synpuf query  \
0                                                                                                                                                                                          select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;   
1                                                                                                                                                                                           select count(distinct desynpuf_id) from rwdex_raw_synpuf.inpatient_claims;   
2                                                                                                                                           select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'   
3                                                                                                                                          select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'   
4  select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims \nwhere clm_from_dt between '2015-01-01' and '2016-01-01' \nunion select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims\nwhere clm_admsn_dt between '2015-01-01' and '2016-01-01'    

                               Description  Difficulty Genreralization  \
0          count the number of outpatients         1.0             NaN   
1           count the number of inpatients         1.0             NaN   
2       select all inpatients in year 2005         2.0             Yes   
3      select all outpatients in year 2006         2.0             Yes   
4  return total patients from 2015 to 2016         3.0             NaN   

  Input table  \
0         NaN   
1         NaN   
2         NaN   
3         NaN   
4         NaN   

                                                                                                                                                      GPT-4 response  \
0                                                                                      SELECT COUNT(*) AS outpatient_count\nFROM rwdex_raw_synpuf.outpatient_claims;   
1                                                            SELECT COUNT(DISTINCT desynpuf_id) AS distinct_inpatient_count\nFROM rwdex_raw_synpuf.inpatient_claims;   
2                          SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_synpuf.inpatient_claims\nWHERE clm_admsn_dt >= '2005-01-01' AND clm_admsn_dt <= '2005-12-31';   
3                           SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_synpuf.outpatient_claims\nWHERE clm_from_dt >= '2006-01-01' AND clm_from_dt <= '2006-12-31';   
4  SELECT COUNT(DISTINCT desynpuf_id) AS total_patients\nFROM rwdex_raw_synpuf.outpatient_claims\nWHERE clm_from_dt >= '2015-01-01' AND clm_from_dt <= '2016-12-31';   

                                                    Comment   \
0                    I want the distinct numbers of patients   
1                                                        NaN   
2                                                        NaN   
3                                                        NaN   
4  I want results from two tables: outpatient and inpatients   

                                                                                                                                                                                                                                                                                                                                                

In [7]:
#Check generalization column
df.Genreralization.unique()

array([nan, 'Yes', 'code_list', 'code_list_1, code_list_2'], dtype=object)

In [8]:
df[df.Genreralization=='Yes']

Table reference  \
2              t3   
3              t4   
5              t6   
8              t9   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Synpuf query  \
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'   
3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'   
5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where admtng_icd9_dgns_cd in (select icd9_cd from diabetes_code_list)   
8  select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where \nicd9_dgns_cd_1 in (select icd9_cd from diabetes_code_list) or\nicd9_dgns_cd_2 in (select icd9_cd from diabetes_code_list) or \nicd9_dgns_cd_3 in (select icd9_cd from diabetes_code_list) or \nicd9_dgns_cd_4 in (select icd9_cd from diabetes_code_list) or\nicd9_dgns_cd_5 in (select icd9_cd from diabetes_code_list) or \nicd9_dgns_cd_6 in (select icd9_cd from diabetes_code_list) or\nicd9_dgns_cd_7 in (select icd9_cd from diabetes_code_list) or\nicd9_dgns_cd_8 in (select icd9_cd from diabetes_code_list) or\nicd9_dgns_cd_9 in (select icd9_cd from diabetes_code_list) or \nicd9_dgns_cd_10 in (select icd9_cd from diabetes_code_list)   

                                                             Description  \
2                                     select all inpatients in year 2005   
3                                    select all outpatients in year 2006   
5  select the inpatients who receive hospitalization because of diabetes   
8                 select t

In [9]:
df[df.Genreralization=='code_list']

Table reference  \
19             t20   
20             t21   
23             t24   
25             t26   
32             t33   
38             t39   
39             t40   

                                                                                                                                                                                                                                                                   Synpuf query  \
19                                             select distinct a.desynpuf_id from rwdex_raw_synpuf.inpatient_claims a inner join rwdex_raw_synpuf.beneficiary_summary b on a.desynpuf_id = b.desynpuf_id and b.bene_death_dt is not null and a.admtng_icd9_dgns_cd in code_list   
20                                                                                                                        select distinct desynpuf_id, min(clm_admsn_dt) from rwdex_raw_synpuf.inpatient_claims a where a.admtng_icd9_dgns_cd in code_list group by desynpuf_id   
23  select distinct a.desynpuf_id, timestampdiff(year, b.birth_dt, a.min(clm_admsn_dt))  as age from rwdex_raw_synpuf.inpatient_claims inner join rwdex_raw_synpuf.beneficiary_summary b on a.desynpuf_id = b.desynpuf_id and a.admtng_icd10_dgns_cd in code_list and age >= 30   
25                                                     select distinct a.desynpuf_id from rwdex_raw_synpuf.inpatient_claims a inner join rwdex_raw_synpuf.beneficiary_summary b on a.desynpuf_id = b.desynpuf_id and b.bene_race_cd = 1 and a.admtng_icd10_dgns_cd in code_list   
32                                                                                                                      select desynpuf_id, count(*) as cnt from rwdex_raw_synpuf.inpatient_claims where admtng_icd10_dgns_cd in code_list group by desynpuf_id having cnt >= 3   
38                            select desynpuf_id, min(clm_from_dt), max(clm_through_dt), datediff(day, max(clm_through_dt), clm_from_dt) as duration from rwdex_raw_synpuf.inpatient_claims where admtng_icd10_dgns_cd in code_list group by desynpuf_id having duration >= 365   
39    select a.desynpuf_id, min(a.clm_from_dt) dt1 from rwdex_raw_synpuf.outpatient_claims a \nwhere admtng_icd10_dgns_cd in code_list group by desynpuf_id \nhaving a.desynpuf_id in (select desynpuf_id from rwdex_raw_synpuf.outpatient_claims b where dt1 < b.clm_admsn_dt)   

                                                                                                          Description  \
19                                 select the inpatients who have died with diagnosis of diseases listed in code_list   
20                                          select the earliest diagnosis date of disease in code_list for inpatients   
23                           select inpatients who are at least 30 years old when diagnosed with disease in code_list   
25                                                     select white inpatients with diagnosis of disease in code_list   
32                       select inpatients who have at least three hospitalization visits due to disease in code_list   
38  select the duration of treatment for inpatients who are diagnosed with disease in code_list of more than one year   
39             select the outpatients who are diagnosed with disease in code_list and are sent to hospital thereafter   

    Difficulty Genreralization Input table  \
19         4.0       code_list         NaN   
20         3.0       code_list         NaN   
23         4.0       code_list         NaN   
25         3.0       code_list         NaN   
32         3.0       code_list         NaN   
38         3.0       code_list         NaN   
39         4.0       code_list         NaN   

                                                                                                                                                                                                                                                                                               

In [10]:
df[df.Genreralization=='code_list_1, code_list_2']

Table reference  \
21             t22   

                                                                                                                                                                                                                                                                                                  Synpuf query  \
21  select distinct a.desynpuf_id, a.min(clm_admsn_dt) dt1, b.min(clm_admsn_dt) dt2 from rwdex_raw_synpuf.inpatient_claims a inner join rwdex_raw_synpuf.inpatient_claims b on a.desynpuf_id = b.desynpuf_id and a.admtng_icd10_dgns_cd in code_list_1 and b.admtng_icd10_dgns_cd in code_list_2 and dt1 < dt2   

                                                                                              Description  \
21  select patients who are diagnosed disease in code_list_1 prior to diagnosis of disease in code_list_2   

    Difficulty           Genreralization Input table  \
21         4.0  code_list_1, code_list_2         NaN   

                                                                                                                                                                                                                                                                                                                                         GPT-4 response  \
21  SELECT DISTINCT i1.desynpuf_id\nFROM rwdex_raw_synpuf.inpatient_claims i1\nINNER JOIN rwdex_raw_synpuf.inpatient_claims i2 ON i1.desynpuf_id = i2.desynpuf_id\nWHERE i1.icd9_dgns_cd_1 IN ('DiseaseCode1_1', 'DiseaseCode1_2', ...)\n  AND i2.icd9_dgns_cd_1 IN ('DiseaseCode2_1', 'DiseaseCode2_2', ...)\n  AND i1.clm_admsn_dt < i2.clm_admsn_dt;   

                                                Comment   \
21  code_list is a given file, with column name 'icd-10'   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Revised GPT-4 response  
21  SELECT DISTINCT i1.desynpuf_id\nFROM rwdex_raw_synpuf.inpatient_claims i1\nWHERE EXISTS (\n  SELECT 1\n  FROM code_list code_list_1\n  WHERE (i1.icd9_dgns_cd_1 = code_list_1.icd10\n         OR i1.icd9_dgns_cd_2 = code_list_1.icd10\n         OR i1.icd9_dgns_cd_3 = code_list_1.icd10\n         -- Add more OR conditions for additional diagnosis codes in code_list_1\n        )\n) AND EXISTS (\n  SELECT 1\n  FROM rwdex_raw_synpuf.inpatient_claims i2\n  INNER JOIN code_list code_list_2 ON i2.icd9_dgns_cd_1 = code_list_2.icd10\n                                OR i2.icd9_dgns_cd_2 = code_list_2.icd10\n                                OR i2.icd9_dgns_cd_3 = code_list_2.icd10\n                                -- Add more OR conditions for additional diagnosis codes in code_list_2\n  WHERE i1.desynpuf_id = i2.desynpuf_id\n    AND i1.clm_admsn_dt < i2.clm_admsn_dt\n);

In [11]:
df = df.rename(columns=COLUMN_MAPPINGS)
df.head()

qa_id  \
0    t1   
1    t2   
2    t3   
3    t4   
4    t5   

                                                                                                                                                                                                                                                                 query  \
0                                                                                                                                                                                          select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;   
1                                                                                                                                                                                           select count(distinct desynpuf_id) from rwdex_raw_synpuf.inpatient_claims;   
2                                                                                                                                           select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'   
3                                                                                                                                          select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'   
4  select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims \nwhere clm_from_dt between '2015-01-01' and '2016-01-01' \nunion select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims\nwhere clm_admsn_dt between '2015-01-01' and '2016-01-01'    

                                  question  difficulty_level code_list_needed  \
0          count the number of outpatients               1.0              NaN   
1           count the number of inpatients               1.0              NaN   
2       select all inpatients in year 2005               2.0              Yes   
3      select all outpatients in year 2006               2.0              Yes   
4  return total patients from 2015 to 2016               3.0              NaN   

  code_list_table  \
0             NaN   
1             NaN   
2             NaN   
3             NaN   
4             NaN   

                                                                                                                                                       gpt4_response  \
0                                                                                      SELECT COUNT(*) AS outpatient_count\nFROM rwdex_raw_synpuf.outpatient_claims;   
1                                                            SELECT COUNT(DISTINCT desynpuf_id) AS distinct_inpatient_count\nFROM rwdex_raw_synpuf.inpatient_claims;   
2                          SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_synpuf.inpatient_claims\nWHERE clm_admsn_dt >= '2005-01-01' AND clm_admsn_dt <= '2005-12-31';   
3                           SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_synpuf.outpatient_claims\nWHERE clm_from_dt >= '2006-01-01' AND clm_from_dt <= '2006-12-31';   
4  SELECT COUNT(DISTINCT desynpuf_id) AS total_patients\nFROM rwdex_raw_synpuf.outpatient_claims\nWHERE clm_from_dt >= '2015-01-01' AND clm_from_dt <= '2016-12-31';   

                                         gpt4_followup_input  \
0                    I want the distinct numbers of patients   
1                                                        NaN   
2                                                        NaN   
3                                                        NaN   
4  I want results from two tables: outpatient and inpatients   

                                                                                                                                                                                                                                                                                                                                          

In [12]:
df.to_csv(COMBINED_DATA_PATH, index=False)